In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/galaxy-zoo-the-galaxy-challenge/all_ones_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_test_rev1.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/central_pixel_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/all_zeros_benchmark.zip
/kaggle/input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip


In [3]:
!rm -f torch.py
!rm -f torch.pyc
!rm -rf __pycache__


In [7]:
# Only install what's needed
!pip install -q transformers timm datasets

!pip install -q transformers timm

!pip install -q transformers datasets timm
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import pandas as pd

from transformers import AutoFeatureExtractor, AutoModelForImageClassification


In [8]:
import zipfile

# Unzip the training images
with zipfile.ZipFile("/kaggle/input/galaxy-zoo-the-galaxy-challenge/images_training_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall("/kaggle/working/images_training_rev1")

# Unzip the training labels
with zipfile.ZipFile("/kaggle/input/galaxy-zoo-the-galaxy-challenge/training_solutions_rev1.zip", 'r') as zip_ref:
    zip_ref.extractall("/kaggle/working/")


In [9]:
DATA_DIR = "/kaggle/working/images_training_rev1"
LABELS_PATH = "/kaggle/working/training_solutions_rev1.csv"


In [10]:
class GalaxyZooDataset(Dataset):
    def __init__(self, image_dir, labels_csv, transform=None):
        self.image_dir = image_dir
        self.df = pd.read_csv(labels_csv)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = int(row["GalaxyID"])  # ✅ Fix here
        image_path = os.path.join(self.image_dir, "images_training_rev1", f"{image_id}.jpg")

        image = Image.open(image_path).convert("RGB")
        label = torch.tensor(row[1:].values, dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label


In [11]:
image_dir = "/kaggle/working/images_training_rev1"  # or whatever your correct path is
image_id = 100008  # for example
image_path = os.path.join(image_dir, f"{image_id}.jpg")


In [12]:
extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=extractor.image_mean, std=extractor.image_std)
])

test_dataset = GalaxyZooDataset(DATA_DIR, LABELS_PATH, transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

2025-04-16 11:27:00.575473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744802820.760449      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744802820.815804      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/transformers/models/convnext/feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


In [13]:
import torch
print(torch.__version__)
print("GPU available:", torch.cuda.is_available())


2.5.1+cu124
GPU available: True


In [14]:
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-50")
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [16]:
from sklearn.metrics import accuracy_score
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
all_preds = []
all_labels = []

model.eval()  # Just in case

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)  # Make sure model returns logits
        logits = outputs.logits  # This should be [batch_size, num_classes]

        all_preds.append(logits.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

# Combine batches correctly
y_pred = np.concatenate(all_preds, axis=0)  # shape: [num_samples, num_classes]
y_true = np.concatenate(all_labels, axis=0)

# Fix shape of true labels
if y_true.ndim > 1 and y_true.shape[1] > 1:
    y_true = y_true.argmax(axis=1)  # for one-hot
else:
    y_true = y_true.squeeze()

# Convert logits to predicted class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate Accuracy
accuracy = accuracy_score(y_true, y_pred_labels)
print(f"Accuracy on test set: {accuracy:.4f}")


100%|██████████| 1925/1925 [05:09<00:00,  6.22it/s]

Accuracy on test set: 0.0001
